In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame

# Specify the file path
file_path = "../Data/prosperLoanData.csv"

In [3]:
# Read the CSV file into a Pandas DataFrame
Uncleaned_df = pd.read_csv(file_path)

# Review the DataFrame

Uncleaned_df

,ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
0,1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0.0,0.0,0.0,0.0,1.0,0,0,0.0,258
1,10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,NaN,36,Current,NaN,0.12016,0.0920,0.0820,...,0.00,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
2,0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0.0,0.0,0.0,0.0,1.0,0,0,0.0,41
3,0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,NaN,36,Current,NaN,0.12528,0.0974,0.0874,...,-108.01,0.0,0.0,0.0,0.0,1.0,0,0,0.0,158
4,0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,NaN,36,Current,NaN,0.24614,0.2085,0.1985,...,-60.27,0.0,0.0,0.0,0.0,1.0,0,0,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,E6D9357655724827169606C,753087,2013-04-14 05:55:02.663000000,NaN,36,Current,NaN,0.22354,0.1864,0.1764,...,-75.58,0.0,0.0,0.0,0.0,1.0,0,0,0.0,1
113933,E6DB353036033497292EE43,537216,2011-11-03 20:42:55.333000000,NaN,36,FinalPaymentInProgress,NaN,0.13220,0.1110,0.1010,...,-30.05,0.0,0.0,0.0,0.0,1.0,0,0,0.0,22
113934,E6E13596170052029692BB1,1069178,2013-12-13 05:49:12.703000000,NaN,60,Current,NaN,0.23984,0.2150,0.2050,...,-16.91,0.0,0.0,0.0,0.0,1.0,0,0,0.0,119
113935,E6EB3531504622671970D9E,539056,2011-11-14 13:18:26.597000000,NaN,60,Completed,2013-08-13 00:00:00,0.28408,0.2605,0.2505,...,-235.05,0.0,0.0,0.0,0.0,1.0,0,0,0.0,274


In [4]:
#Creating Credit Score ( Lowest + Highest credit score /2 )
Uncleaned_df['Average Credit Score'] = (Uncleaned_df['CreditScoreRangeLower'] + Uncleaned_df['CreditScoreRangeUpper']) / 2


In [5]:
## CLEAN UP DATA 


In [6]:
#types of columns 
specific_columns = ['Term','LoanStatus','ProsperRating (numeric)','Average Credit Score','BorrowerState','Occupation','EmploymentStatus','IsBorrowerHomeowner','IncomeRange','StatedMonthlyIncome','EstimatedLoss','LP_GrossPrincipalLoss',
'BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','RevolvingCreditBalance','LoanOriginalAmount','ProsperPrincipalBorrowed','ProsperPrincipalOutstanding','TotalProsperPaymentsBilled', 'OnTimeProsperPayments', 'TotalProsperLoans', 'PublicRecordsLast10Years']  




In [7]:
Uncleaned_df = Uncleaned_df[specific_columns].copy()
Uncleaned_df.head()

,Term,LoanStatus,ProsperRating (numeric),Average Credit Score,BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,StatedMonthlyIncome,...,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,TotalProsperPaymentsBilled,OnTimeProsperPayments,TotalProsperLoans,PublicRecordsLast10Years
0,36,Completed,NaN,649.5,CO,Other,Self-employed,True,"$25,000-49,999",3083.333333,...,78,330.43,0.0,9425,NaN,NaN,NaN,NaN,NaN,0.0
1,36,Current,6.0,689.5,CO,Professional,Employed,False,"$50,000-74,999",6125.000000,...,0,318.93,3989.0,10000,NaN,NaN,NaN,NaN,NaN,1.0
2,36,Completed,NaN,489.5,GA,Other,Not available,False,Not displayed,2083.333333,...,86,123.32,NaN,3001,NaN,NaN,NaN,NaN,NaN,0.0
3,36,Current,6.0,809.5,GA,Skilled Labor,Employed,True,"$25,000-49,999",2875.000000,...,16,321.45,1444.0,10000,NaN,NaN,NaN,NaN,NaN,0.0
4,36,Current,3.0,689.5,MN,Executive,Employed,True,"$100,000+",9583.333333,...,6,563.97,6193.0,15000,11000.0,9947.9,11.0,11.0,1.0,0.0


In [9]:
specific_columns_uncleaned_df = Uncleaned_df[specific_columns]
specific_columns_uncleaned_df.head(20)

,Term,LoanStatus,ProsperRating (numeric),Average Credit Score,BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,StatedMonthlyIncome,...,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,TotalProsperPaymentsBilled,OnTimeProsperPayments,TotalProsperLoans,PublicRecordsLast10Years
0,36,Completed,NaN,649.5,CO,Other,Self-employed,True,"$25,000-49,999",3083.333333,...,78,330.43,0.0,9425,NaN,NaN,NaN,NaN,NaN,0.0
1,36,Current,6.0,689.5,CO,Professional,Employed,False,"$50,000-74,999",6125.000000,...,0,318.93,3989.0,10000,NaN,NaN,NaN,NaN,NaN,1.0
2,36,Completed,NaN,489.5,GA,Other,Not available,False,Not displayed,2083.333333,...,86,123.32,NaN,3001,NaN,NaN,NaN,NaN,NaN,0.0
3,36,Current,6.0,809.5,GA,Skilled Labor,Employed,True,"$25,000-49,999",2875.000000,...,16,321.45,1444.0,10000,NaN,NaN,NaN,NaN,NaN,0.0
4,36,Current,3.0,689.5,MN,Executive,Employed,True,"$100,000+",9583.333333,...,6,563.97,6193.0,15000,11000.0,9947.90,11.0,11.0,1.0,0.0
5,60,Current,5.0,749.5,NM,Professional,Employed,True,"$100,000+",8333.333333,...,3,342.37,62999.0,15000,NaN,NaN,NaN,NaN,NaN,0.0
6,36,Current,2.0,689.5,KS,Sales - Retail,Employed,False,"$25,000-49,999",2083.333333,...,11,122.67,5812.0,3000,NaN,NaN,NaN,NaN,NaN,0.0
7,36,Current,4.0,709.5,CA,Laborer,Employed,False,"$25,000-49,999",3355.750000,...,10,372.60,1260.0,10000,NaN,NaN,NaN,NaN,NaN,1.0
8,36,Current,7.0,829.5,IL,Food Service,Employed,True,"$25,000-49,999",3333.333333,...,3,305.54,9906.0,10000,NaN,NaN,NaN,NaN,NaN,0.0
9,36,Current,7.0,829.5,IL,Food Service,Employed,True,"$25,000-49,999",3333.333333,...,3,305.54,9906.0,10000,NaN,NaN,NaN,NaN,NaN,0.0


In [10]:
#changing data that is not a value to be NA
specific_columns_uncleaned_df.replace("Not displayed", pd.NA, inplace=True)

specific_columns_uncleaned_df

,Term,LoanStatus,ProsperRating (numeric),Average Credit Score,BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,StatedMonthlyIncome,...,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,TotalProsperPaymentsBilled,OnTimeProsperPayments,TotalProsperLoans,PublicRecordsLast10Years
0,36,Completed,NaN,649.5,CO,Other,Self-employed,True,"$25,000-49,999",3083.333333,...,78,330.43,0.0,9425,NaN,NaN,NaN,NaN,NaN,0.0
1,36,Current,6.0,689.5,CO,Professional,Employed,False,"$50,000-74,999",6125.000000,...,0,318.93,3989.0,10000,NaN,NaN,NaN,NaN,NaN,1.0
2,36,Completed,NaN,489.5,GA,Other,Not available,False,<NA>,2083.333333,...,86,123.32,NaN,3001,NaN,NaN,NaN,NaN,NaN,0.0
3,36,Current,6.0,809.5,GA,Skilled Labor,Employed,True,"$25,000-49,999",2875.000000,...,16,321.45,1444.0,10000,NaN,NaN,NaN,NaN,NaN,0.0
4,36,Current,3.0,689.5,MN,Executive,Employed,True,"$100,000+",9583.333333,...,6,563.97,6193.0,15000,11000.0,9947.90,11.0,11.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113932,36,Current,4.0,709.5,IL,Food Service Management,Employed,True,"$50,000-74,999",4333.333333,...,11,364.74,7714.0,10000,NaN,NaN,NaN,NaN,NaN,1.0
113933,36,FinalPaymentInProgress,6.0,709.5,PA,Professional,Employed,True,"$75,000-99,999",8041.666667,...,28,65.57,15743.0,2000,33501.0,4815.42,60.0,60.0,3.0,0.0
113934,60,Current,3.0,709.5,TX,Other,Employed,True,"$25,000-49,999",2875.000000,...,3,273.35,22147.0,10000,NaN,NaN,NaN,NaN,NaN,0.0
113935,60,Completed,4.0,689.5,GA,Food Service,Full-time,True,"$25,000-49,999",3875.000000,...,28,449.55,11956.0,15000,5000.0,3264.37,16.0,16.0,1.0,1.0


In [11]:
#drop NaN's
specific_columns_uncleaned_df.dropna()

,Term,LoanStatus,ProsperRating (numeric),Average Credit Score,BorrowerState,Occupation,EmploymentStatus,IsBorrowerHomeowner,IncomeRange,StatedMonthlyIncome,...,LoanMonthsSinceOrigination,MonthlyLoanPayment,RevolvingCreditBalance,LoanOriginalAmount,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,TotalProsperPaymentsBilled,OnTimeProsperPayments,TotalProsperLoans,PublicRecordsLast10Years
4,36,Current,3.0,689.5,MN,Executive,Employed,True,"$100,000+",9583.333333,...,6,563.97,6193.0,15000,11000.0,9947.90,11.0,11.0,1.0,0.0
16,60,Current,2.0,649.5,CA,Professional,Full-time,True,"$75,000-99,999",8291.666667,...,8,124.76,14030.0,4000,10700.0,4064.87,67.0,67.0,3.0,0.0
19,60,Current,3.0,689.5,OR,Laborer,Employed,False,"$25,000-49,999",3075.000000,...,5,180.45,3140.0,6500,8000.0,5786.00,12.0,12.0,1.0,0.0
33,36,Completed,7.0,769.5,IL,Other,Employed,False,"$100,000+",13083.333333,...,41,500.28,41.0,16000,4000.0,0.01,8.0,8.0,1.0,1.0
47,36,Current,6.0,689.5,CO,Professional,Employed,True,"$50,000-74,999",5416.666667,...,10,505.34,20812.0,15000,27000.0,11308.14,20.0,20.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113924,60,Current,6.0,649.5,WA,Other,Employed,False,"$25,000-49,999",3208.333333,...,17,118.92,8044.0,5000,9300.0,5094.62,38.0,35.0,2.0,0.0
113928,36,Completed,4.0,749.5,CO,Other,Full-time,False,"$25,000-49,999",2333.333333,...,33,73.30,4.0,2000,14250.0,0.02,47.0,47.0,5.0,0.0
113930,36,Current,5.0,689.5,IN,Professional,Employed,True,"$75,000-99,999",6250.000000,...,8,106.05,3245.0,3000,2000.0,889.75,7.0,7.0,1.0,0.0
113933,36,FinalPaymentInProgress,6.0,709.5,PA,Professional,Employed,True,"$75,000-99,999",8041.666667,...,28,65.57,15743.0,2000,33501.0,4815.42,60.0,60.0,3.0,0.0


In [12]:
specific_columns_uncleaned_df.dtypes

Term                             int64
LoanStatus                      object
ProsperRating (numeric)        float64
Average Credit Score           float64
BorrowerState                   object
Occupation                      object
EmploymentStatus                object
IsBorrowerHomeowner               bool
IncomeRange                     object
StatedMonthlyIncome            float64
EstimatedLoss                  float64
LP_GrossPrincipalLoss          float64
BorrowerRate                   float64
EmploymentStatusDuration       float64
LoanMonthsSinceOrigination       int64
MonthlyLoanPayment             float64
RevolvingCreditBalance         float64
LoanOriginalAmount               int64
ProsperPrincipalBorrowed       float64
ProsperPrincipalOutstanding    float64
TotalProsperPaymentsBilled     float64
OnTimeProsperPayments          float64
TotalProsperLoans              float64
PublicRecordsLast10Years       float64
dtype: object

In [16]:
specific_numeric_columns = specific_columns_uncleaned_df[['Term','EstimatedLoss','LP_GrossPrincipalLoss','BorrowerRate','EmploymentStatusDuration','LoanMonthsSinceOrigination','MonthlyLoanPayment','LoanOriginalAmount']]

specific_numeric_columns.head()


,Term,EstimatedLoss,LP_GrossPrincipalLoss,BorrowerRate,EmploymentStatusDuration,LoanMonthsSinceOrigination,MonthlyLoanPayment,LoanOriginalAmount
0,36,NaN,0.0,0.1580,2.0,78,330.43,9425
1,36,0.0249,0.0,0.0920,44.0,0,318.93,10000
2,36,NaN,0.0,0.2750,NaN,86,123.32,3001
3,36,0.0249,0.0,0.0974,113.0,16,321.45,10000
4,36,0.0925,0.0,0.2085,44.0,6,563.97,15000


In [14]:
# Initialize the StandardScaler
Scaled_columns = StandardScaler().fit_transform(
    specific_numeric_columns
)

In [15]:
# Fit and transform the selected columns
Scaled_df = pd.DataFrame(Scaled_columns)
Scaled_df.head()

#Columns not included
# 'LoanStatus','ProsperRating (Alpha)','BorrowerState','Occupation','EmploymentStatus','IsBorrowerHomeowner','IncomeRange'



,0,1,2,3,4,5,6,7
0,-0.462837,NaN,-0.293257,-0.464648,-0.995676,1.538101,0.300753,0.174196
1,-0.462837,-1.184816,-0.293257,-1.346787,-0.551138,-1.064150,0.241074,0.266258
2,-0.462837,NaN,-0.293257,1.099145,NaN,1.804998,-0.774043,-0.854340
3,-0.462837,-1.184816,-0.293257,-1.274612,0.179174,-0.530355,0.254151,0.266258
4,-0.462837,0.260763,-0.293257,0.210323,-0.551138,-0.863977,1.512708,1.066799


In [ ]:
# Header
# ListingKey
# Term
# LoanStatus
# ProsperRating (Alpha)
# BorrowerState
# Occupation
# EmploymentStatus
# IsBorrowerHomeowner
# IncomeRange
# EstimatedLoss
# LP_GrossPrincipalLoss
# BorrowerRate
# EmploymentStatusDuration
# LoanMonthsSinceOrigination
# MonthlyLoadPayment
# LoanOriginalAmount